In [1]:
import pandas as pd
from collections import Counter
import networkx as nx 
from networkx.readwrite import json_graph
import json
import scipy
import numpy as np

In [51]:
event_name = 'TEDxNations'

In [3]:
event_data = pd.read_table('data/[%s]_data_clean_features.txt' % event_name, sep='\t', header=0, encoding='utf-8')

In [65]:
#compute top tokens

topk = 50

def gather_tokens(data):
    all_tokens = []
    for doc in data:
        tokens = doc.split()
        all_tokens.extend(tokens)
    return all_tokens

#gather tokens from all docs 
all_tokens = gather_tokens(event_data['text_clean_tokens'])
#create counter object
tokens_cntr = Counter(all_tokens)

top_tokens = [token for token, count in tokens_cntr.most_common(topk)]

In [66]:
top_tokens = [token for token in top_tokens if token != event_name.lower()]

In [67]:
#manually construct scipy sparse matrix - alternative to sklearn vectorizer
vocabulary = {} #terms with indices
data = [] #1
row = [] #docs
col = [] #terms

for i,doc in event_data.iterrows():
    for term in doc['text_clean_tokens'].split():
        # get column index, adding the term to the vocabulary if needed
        j = vocabulary.setdefault(term, len(vocabulary))
        data.append(1) # uniform weights
        row.append(i)
        col.append(j)

A = scipy.sparse.coo_matrix((data, (row, col)))

In [68]:
#compute co occurrences matrix (terms by terms)
X = A.T * A

In [69]:
with open('data/d3matrix_termsco_datacols.txt', 'w') as f:
    for term in top_tokens:
        f.write(term+'\n') 

In [70]:
header = ['term1', 'term2', 'value']
h = pd.DataFrame(columns=header)
h.to_csv('data/d3matrix_termsco_data.tsv', mode='w', sep='\t', encoding='utf-8', index=False, header=True)

t1 = 1
for term1 in top_tokens:
    index1 = vocabulary.get(term1)
    t2 = 1
    for term2 in top_tokens:
        index2 = vocabulary.get(term2)
        co = (X[index1, index2] if term2 != term1 else 0) 
        r = [t1, t2, co]
        row = pd.Series(r, index=header)
        row = pd.DataFrame(r).T
        row.to_csv('data/d3matrix_termsco_data.tsv', mode='a', sep='\t', encoding='utf-8', index=False, header=False)
        t2+=1
    t1+=1